In [17]:
import re
import numpy as np
from sklearn import datasets
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier

class SVM(object):

    def __init__(self, training_dataset_, test_dataset_):
        self.training_dataset = training_dataset_
        self.test_dataset = test_dataset_
        self.classes = {}
        self.X_train = None
        self.Y_train = None
        self.X_test = None
        self.Y_test = None

        self.support_indecies = None
        self.train_errors = None
        self.val_errors = None
        self.train_loss = None
        self.val_loss = None
        
    def read_data(self):
        f = open(self.training_dataset, 'r')
        rows = list(re.split(' ', row) for row in re.split('\n', f.read())[:-1])
        names, self.Y_train = np.unique(list(row[-1] for row in rows), return_inverse=True)
        self.X_train = np.empty((0,4), float)
        f.close()
        for row in rows:
            self.X_train = np.append(self.X_train, np.array([np.array(row[:-1]).astype(float)]), axis = 0)
        f = open(self.test_dataset, 'r')
        f.close()
        
        f = open(self.test_dataset, 'r')
        rows = list(re.split(' ', row) for row in re.split('\n', f.read())[:-1])
        names, self.Y_test = np.unique(list(row[-1] for row in rows), return_inverse=True)
        self.X_test = np.empty((0,4), float)
        f.close()
        for row in rows:
            self.X_test = np.append(self.X_test, np.array([np.array(row[:-1]).astype(float)]), axis = 0)
        

    '''
        Using one-vs-rest strategy using SVC via svm.SVC. 
        Output the train_loss, test_loss, and index of support vectors
    '''    
    def SVM(self, C, kernel = "linear", degree = 1, gamma = 1):
        # Since use one-vs-rest strategy here, we mannually label the "rest"
        Y_train_for_0 = []
        Y_train_for_1 = []
        Y_train_for_2 = []
        for i in self.Y_train:
            if (i == 0):
                Y_train_for_0.append(i)
                Y_train_for_1.append(-1)
                Y_train_for_2.append(-1)
            elif (i == 1):
                Y_train_for_0.append(-1)
                Y_train_for_1.append(i)
                Y_train_for_2.append(-1)
            elif (i == 2):
                Y_train_for_0.append(-1)
                Y_train_for_1.append(-1)
                Y_train_for_2.append(i)
        Y_test_for_0 = []
        for i in self.Y_test:
            if (i == 0):
                Y_test_for_0.append(0)
            elif (i != 0):
                Y_test_for_0.append(-1)

        # implement using SVC
        clf_for_0 = SVC(C= C, kernel = kernel, decision_function_shape = "ovo", degree = degree, gamma = gamma)
        clf_for_1 = SVC(C= C, kernel = kernel, decision_function_shape = "ovo", degree = degree, gamma = gamma)
        clf_for_2 = SVC(C= C, kernel = kernel, decision_function_shape = "ovo", degree = degree, gamma = gamma)
        

        model_for_0 = clf_for_0.fit(self.X_train, Y_train_for_0)
        model_for_1 = clf_for_1.fit(self.X_train, Y_train_for_1)
        model_for_2 = clf_for_2.fit(self.X_train, Y_train_for_2)

        # calcualting the training_loss
        train_predict = ovr_predict(clf_for_0, clf_for_1, clf_for_2, self.X_train)
        self.train_loss = get_error_rate(train_predict, Y_train_for_0)

        # calculate the test_loss
        test_predict = ovr_predict(clf_for_0, clf_for_1, clf_for_2, self.X_test)
        self.val_loss = get_error_rate(test_predict, Y_test_for_0)

        # get the support vector
        # list(set()): dedupe the same data
        self.support_indecies = list(set(model_for_0.support_.tolist() + model_for_1.support_.tolist() + model_for_2.support_.tolist()))
        support_vector_index_0 = model_for_0.support_.tolist()
        support_vector_index_1 = model_for_1.support_.tolist()
        support_vector_index_2 = model_for_2.support_.tolist()

        if (kernel == "linear"):
            w_0 = model_for_0.coef_
            w_1 = model_for_1.coef_
            w_2 = model_for_2.coef_

        b_0 = model_for_0.intercept_
        b_1 = model_for_1.intercept_
        b_2 = model_for_2.intercept_

        if (kernel == "linear"):
            return self.train_loss, self.val_loss, support_vector_index_0, support_vector_index_1, support_vector_index_2, w_0, b_0, w_1, b_1, w_2, b_2
        else:    # without return the w
            return self.train_loss, self.val_loss, support_vector_index_0, support_vector_index_1, support_vector_index_2, b_0, b_1, b_2
    
    def SVM_slack(self, C, kernel = "linear"):
    
        training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
        support_vector_index_virginica, w_0, b_0, w_1, b_1, w_2, b_2 = svm.SVM(C = C, kernel = "linear")

        return training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
        support_vector_index_virginica, w_0, b_0, w_1, b_1, w_2, b_2
    
    def SVM_kernel_poly2(self, C = 1, kernel = "poly", degree = 2):
    
        training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2 = svm.SVM(C = C, kernel = kernel, degree = degree, gamma = 1)

        return training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2         
        
    
    def SVM_kernel_poly3(self, C = 1, kernel = "poly", degree = 3):
    
        training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2 = svm.SVM(C = C, kernel = kernel, degree = degree, gamma = 1)

        return training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2   
    
    def SVM_kernel_rbf(self, C, kernel = "rbf", gamma = 1/2):
    
        training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2 = svm.SVM(C = C, kernel = kernel, gamma = gamma)

        return training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2
        
    
    def SVM_kernel_sigmoid(self, C, kernel = "sigmoid", gamma = 1/4):
    
        training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2 = svm.SVM(C = C, kernel = kernel, gamma = gamma)

        return training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
            support_vector_index_virginica, b_0, b_1, b_2   


'''
    The predict function for the Multi-class SVM:
    For each sample, find the best score among the three SVM, 
    and adopts the corresponding prediction as final prediction.
'''
def ovr_predict(clf_for_0, clf_for_1, clf_for_2, X):

    predict_0 = clf_for_0.decision_function(X)
    predict_1 = 0
    predict_2 = 0
    
    predict_result = []

    # find the best score of all the three SVMs. The label with best score, should be the predict score
    for i in range(0, len(X)):
        max_scoce = max(predict_0[i],0)
        if (max_scoce == predict_0[i]): 
            predict_result.append(0)
        else: 
            predict_result.append(-1)        

    return predict_result


'''
    auxilary function
    given the classifer: predict_result, ground_truth: X, Y
    return error_rate = miscalssification_number/total_number
'''
def get_error_rate(predict_result, Y):

    error_number = 0
    for i in range(0, len(Y)):
        if predict_result[i] != Y[i]:
            error_number += 1
    error_rate = error_number / len(Y)
    return error_rate

def write_to_file(file, training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
        support_vector_index_virginica, b_0 ,b_1, b_2, w_0 = [], w_1 = [], w_2 = []):
    not_linear = False
    if (len(w_0) == 0 and len(w_1) == 0 and len(w_2) == 0):
        not_linear = True

    with open(file,'w') as f:   
        f.write("training_error: " + str(training_error) + "\n")
        f.write("testing_error: " + str(testing_error) + "\n")
        if not (not_linear):
            f.write("w_of_setosa: " + str(w_0) + "\n")
        f.write("b_of_setosa: " + str(b_0) + "\n")
        f.write("support_vector_indices_of_setosa: " + str(support_vector_index_setosa) + "\n")
        if not (not_linear):
            f.write("w_of_versicolor: " + str(w_1) + "\n")
        f.write("b_of_versicolor" + str(b_1) + "\n")
        f.write("support_vector_indices_of_versicolor: " + str(support_vector_index_versicolor) + "\n")
        if not (not_linear):
            f.write("w_of_virginica: " + str(w_2) + "\n")
        f.write("w_of_virginica: " + str(b_2) + "\n")
        f.write("support_vector_indices_of_virginica: " + str(support_vector_index_virginica) + "\n") 

In [19]:
svm = SVM("train.txt", "test.txt")
svm.read_data()



# problem 2: using SVM with slack variables, for C = 0.1, 0.2, ... , 1.0
C_slack = 0.00000001
training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
    support_vector_index_virginica, w_0, b_0, w_1, b_1, w_2, b_2 = svm.SVM_slack(C = C_slack)
write_to_file("SVM_slack_" + str(C_slack) + ".txt", training_error, testing_error, support_vector_index_setosa, support_vector_index_versicolor, \
    support_vector_index_virginica, b_0, b_1, b_2, w_0, w_1, w_2)

